In [2]:
import PyPDF2
import os

#pdf读取文本
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

def extract_text_from_pdfs(pdf_dir):
    pdf_texts = {}
    for filename in os.listdir(pdf_dir):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, filename)
            pdf_texts[filename] = extract_text_from_pdf(pdf_path)
    return pdf_texts

#文本截断为小chunk
def split_into_chunks(text, max_chunk_size=512):
    #以句号拆分
    for char in ['\n', '\t', '.']:
        text = text.replace(char, '')
    sentences = text.split('。')
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        if len(current_chunk) + len(sentence) < max_chunk_size:
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence 
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

pdf_dir = r'Final_Project_Documents\documents'  #test只有一个较小pdf文件，测试用
pdf_texts = extract_text_from_pdfs(pdf_dir)


预训练模型换成了multi-qa-MiniLM-L6-cos-v1，这个模型更小而且语义匹配的效果更好一点。

In [4]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 加载模型
embedder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1') #或fine_tuned_simcse_model

# 创建FAISS索引
def create_embeddings_and_index(pdf_texts, max_chunk_size=512):
    all_chunks = []
    for filename, text in pdf_texts.items():
        chunks = split_into_chunks(text, max_chunk_size)
        all_chunks.extend(chunks)  # 将所有chunk放入一个列表
    
    embeddings = embedder.encode(all_chunks, convert_to_numpy=True)
    
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    
    return all_chunks, index

all_chunks, index = create_embeddings_and_index(pdf_texts, max_chunk_size=512)

# 搜索函数，检索相似的文段
def search(query, all_chunks, index, k=3):
    query_embedding = embedder.encode([query], convert_to_numpy=True)
    D, I = index.search(query_embedding, k)
    return [(all_chunks[i], D[0][idx]) for idx, i in enumerate(I[0])]




modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

d:\Mypython\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\21102\.cache\huggingface\hub\models--sentence-transformers--multi-qa-MiniLM-L6-cos-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
query = "定位器安装程序，通过导线和电缆的线号选择什么"
retrieved_chunks = search(query, all_chunks, index, k=5)
for chunk, score in retrieved_chunks:
    print(f"Chunk: {chunk}\nScore: {score}\n")

Chunk: 定位器安装程序，通过导线和电缆的线号选择合适的夹接插钉/插孔，将定位器插入锁盘扭转90º， 取下夹接线号选择盘上的保险卡子， 选择合适的夹接档位到SELNO指示线上并到位，按压夹接工具的手柄操作夹接工具必须完成一个夹接循环，待夹接工具的模块到达夹接力矩时防倒转棘轮释放是夹接工具复位
Score: 0.3770199716091156

Chunk: 恢复配电板导线束的固定和捆扎；安装配电系图9-361 EN0313单相线路跳开关结构 统的配电板，摘下ESD防静电外带，摘下所有挂的红色警告牌，闭合线路跳开关，将驾驶舱所有操作过的电门放到正常位 给飞机提供电源对更换的线路跳开关系统进行操作测试，检验线路跳卡关的功能
Score: 0.4064154028892517

Chunk: (c) 对座舱和驾驶舱的下列构件进行检查：(1) 全面地检查是否整洁，是否有可能使控制机构失灵的外物和松动的设备；(2) 座椅和座椅安全带：检查是否处于不良状态和有明显缺陷；(3) 窗和风档：检查是否损坏和破裂；(4) 仪表：检查其状况、座架、标识是否符合要求，和检查其操作性能是否良好；(5) 飞行和发动机控制机构：检查是否安装正确、操作性能良好；(6) 电池：检查是否安装正确、充电恰当；(7) 各种系统：检查是否安装正确和有明显的缺陷，检查它们的一般状况
Score: 0.46906566619873047

Chunk: 将夹接工具的定位器安装在夹接工具上，按照插钉/插孔号选择20号钉位置（红色）并锁定位置， 按照AWG线号22线选择夹接工具的夹接档位  如果使用M22520/1-01夹接工具 （请见图8-387所示）或M22520/2-01夹接工具（请见图8-388所示） ，将插钉/插孔放入夹接工具的定位器，将绝缘去除完成的导线插入插钉/插孔的夹接筒， 按压夹接工具的手柄待到达夹接力矩时， 防倒转棘轮复位反转夹接工具手柄自动释放， 从夹接工具的定位器中取下夹接完成的插钉/插孔
Score: 0.4961960017681122

Chunk: 带安装边的轴有定位销孔，用螺栓和螺帽将螺旋桨固定在轴上 定位销孔让螺旋桨安装在一个位置 有的是预先将带螺纹的圈压入螺栓孔，不再需要螺帽（图11-44） 安装螺旋桨前,，先要检查凸缘有无锈蚀、缺口、毛刺和其它表面缺陷，带螺栓的孔和带螺

换成了gpt-neo-2.7B试了一下，不过这个模型参数量不少，显存消耗也很大（10-12GB）而且效果很大提升，我还是直接找了个api调用了。使用api的话下面的代码块可以忽略

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "EleutherAI/gpt-neo-2.7B"  # gpt2表现很差，可以替换成其他生成模型，如"t5-small", "gpt-3"等，或者到时候直接在线调用某个现成的大模型，代码贴在这里可以应付检查
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

# if torch.cuda.is_available():
#     device = torch.device("cuda")
#     model.to(device)
# else:
#     device = torch.device("cpu")

# 定义generator函数
def generator(input_text, max_length=2048, num_return_sequences=1):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=2048, truncation=True)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,  # 防止重复n-gram
        top_p=0.95,  # 使用nucleus采样
        top_k=50,    # 限制采样的候选数量
        temperature=0.1,  # 温度采样控制生成多样性
        do_sample=True,  # 启用采样
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

def generate_answer(query, retrieved_chunks):
    context = "\n".join([chunk for chunk, _ in retrieved_chunks])
    input_text = f"问题: {query}\n相关信息:\n{context}\n答案:"
    response = generator(input_text, max_length=2048, num_return_sequences=1)
    return response

# 生成答案 ##这gpt2生成的是什么玩意？
# answer = generate_answer(query, retrieved_chunks)
answer = generator("你好",max_length=2048, num_return_sequences=1)
print(answer)


这个key搞了个国内的镜像，不用梯子，我暂时配了10w字符的额度


In [7]:
def generate_input_text(query, retrieved_chunks):
    context = "\n".join([chunk for chunk, _ in retrieved_chunks])
    input_text = f"问题: {query}\n相关信息:\n{context}\n答案:"
    return input_text

from openai import OpenAI
client = OpenAI(
    api_key="sk-fpifp5f293bak59ts7n71sohiqcnivi6qn9v9d08dcnhr9gp",
    base_url="https://api.aihao123.cn/luomacode-api/open-api/v1"
)

response = client.chat.completions.create(
    messages=[
        {'role': 'user', 'content': generate_input_text(query, retrieved_chunks)},
    ],
    model='gpt-3.5-turbo',  # 代码提示上写了可以调用的模型
    stream=True  # 一定要设置True
)

for chunk in response:
    print(chunk.choices[0].delta.content, end="", flush=True)


夹接插钉/插孔时，选择20号钉位置（红色），根据AWG线号22线选择夹接工具的夹接档位。